<a href="https://colab.research.google.com/github/josiahlin119/IMDB-Final-Project/blob/main/Imdb_bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -1 tf-model


Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -1


In [3]:
#Load the libraries
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelBinarizer
from nltk.corpus import stopwords

from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud,STOPWORDS
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize,sent_tokenize
from bs4 import BeautifulSoup
import spacy
import re,string,unicodedata
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.stem import LancasterStemmer,WordNetLemmatizer
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from textblob import TextBlob
from textblob import Word
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
#deep learning

import tensorflow as tf
from tensorflow import keras
from keras.datasets import imdb
from tensorflow.keras import models
from tensorflow.keras import layers

In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [6]:
imdb_data=pd.read_csv('https://raw.githubusercontent.com/josiahlin119/IMDB-Final-Project/main/ReviewData/IMDB%20Dataset.csv')
print(imdb_data.shape)
imdb_data.head

(50000, 2)


<bound method NDFrame.head of                                                   review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]>

In [7]:
imdb_data.describe()

,review,sentiment
count,50000,50000
unique,49582,2
top,Loved today's show!!! It was a variety and not...,positive
freq,5,25000


In [8]:
imdb_data['sentiment'].value_counts()

positive    25000
negative    25000
Name: sentiment, dtype: int64

In [9]:
train_reviews = imdb_data.review[:40000]
train_sentiments = imdb_data.sentiment[:40000]
test_reviews=imdb_data.review[40000:]
test_sentiments=imdb_data.sentiment[40000:]
print(train_reviews.shape, train_sentiments.shape)
print(test_reviews.shape, test_sentiments.shape)
tokenizer=ToktokTokenizer()
#filter out all stop words which do not contribute much to the learning
stopword_list = nltk.corpus.stopwords.words('english')


(40000,) (40000,)
(10000,) (10000,)


Remove irrelevent symbols

In [10]:
def strip_html(text):
  soup = BeautifulSoup(text, "html.parser")
  return soup.get_text()
def remove_between_square_brackets(text):
  return re.sub('\[[^]]*\]','',text)
def remove_special_characters(text, remove_digits=True):
  pattern=r'[^a-zA-Z0-9\s]'
  text=re.sub(pattern,'',text)
  return text
def denoise_text(text):
  text = strip_html(text)
  text = remove_between_square_brackets(text)
  text = remove_special_characters(text)
  return text
imdb_data['review']=imdb_data['review'].apply(denoise_text)


Word Stemming

In [11]:
def my_stemmer(text):
  ps = nltk.porter.PorterStemmer()
  text = ' '.join([ps.stem(word) for word in text.split()])
  return text
imdb_data['review']=imdb_data['review'].apply(my_stemmer)


Remove Stop Words

In [12]:
stop=set(stopwords.words('english'))
print(stop)

def remove_stopwords(text, is_lower_case=False):
  tokens = tokenizer.tokenize(text)
  tokens = [token.strip() for token in tokens]
  if is_lower_case:
    filtered_tokens = [token for token in tokens if token not in stopword_list]
  else:
    filtered_tokens = [token for token in tokens if token.lower() not in stopword_list]
  filtered_text= ' '.join(filtered_tokens)
  return filtered_text
imdb_data['review'] = imdb_data['review'].apply(remove_stopwords)



{"wasn't", 'when', 'now', "shouldn't", "didn't", 'and', 'all', 'with', 'hasn', 'does', 'no', 'then', 'of', 'yours', 'down', 'too', 'whom', 'because', 'being', 'has', "it's", 'nor', 'but', 'my', 'very', 'was', "won't", 'any', 'them', 't', 've', 'are', 'theirs', 'doesn', 'an', 'wasn', "you're", "hadn't", 'did', "needn't", 'until', 'should', 'just', 'once', 'out', 'itself', 'up', 'shan', 'the', 'above', 'than', 'been', 'to', 'they', 'ours', 'which', 'so', 'm', 'her', 'how', 'other', "weren't", 'ourselves', 'shouldn', "isn't", 'most', 'mustn', 'were', 'ain', 'those', 'yourselves', 'themselves', 'on', 's', 'over', 'only', 'what', 'yourself', 'at', "couldn't", 'o', 'couldn', 'through', 'off', 'y', 'she', 'who', 'some', 'herself', 'he', 'had', 'be', 'we', "that'll", 'its', 'hers', 'same', 'by', 'll', 'below', 'again', 'haven', "you'll", "you'd", 'these', 'during', 're', "doesn't", 'will', 'having', 'do', 'doing', "don't", 'your', 'you', 'from', "she's", 'or', 'there', 'after', 'while', 'furth

In [6]:
cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(1,3))
cv_train_reviews=cv.fit_transform(train_reviews)
cv_test_reviews=cv.transform(test_reviews)
print('BOW_cv_train: ', cv_train_reviews.shape)
print('BOW_cv_test:', cv_test_reviews.shape)


NameError: ignored

In [19]:
#Tfidf vectorizer
tv=TfidfVectorizer(min_df=0,max_df=1,use_idf=True,ngram_range=(1,3))
#transformed train reviews
tv_train_reviews=tv.fit_transform(train_reviews)
#transformed test reviews
tv_test_reviews=tv.transform(test_reviews)
print('Tfidf_train:',tv_train_reviews.shape)
print('Tfidf_test:',tv_test_reviews.shape)



Tfidf_train: (40000, 6208633)
Tfidf_test: (10000, 6208633)


In [4]:
max_sequence_length = 100
loss_function = 'binary_crossentropy'
optimizer = 'adam'
addition_metrics = ['accuracy']
number_of_epochs=100
verbosity_mode =True
validation_split = 0.2


Deep Learning Model: Keras. The input data are transformed tokens insteaed of the original words

In [5]:
input_shape = cv_train_reviews.shape
print("input_shape", input_shape)

model = keras.Sequential()

model.add(layers.Dropout(rate=0.5, input_shape=input_shape))
model.add(layers.Conv1D(filters=32, kernel_size=2, padding='same', activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(layers.Flatten())
model.add(layers.Dropout(0.50))
model.add(layers.Dense(1, activation='sigmoid'))

model.compile(optimizer=optimizer, loss=loss_function, metrics=addition_metrics)
model.summary()
history = model.fit(cv_train_reviews, train_sentiments,epochs=number_of_epochs, verbose=verbosity_mode,validation_split=validation_split)
test_results = model.evaluate(cv_test_reviews, test_sentiments, verbose=False)
print(f'Test results - Loss: {test_results[0]} - Accuracy:{100*test_results[1]}')

NameError: ignored

Logistic Regression: Bag of words and Term Frequency-Inverse document Frequency


In [ ]:
lr = LogisticRegression(penalty='l2',max_iter=200, verbose=1, random_state=77)
logistic_bow= lr.fit(cv_train_reviews, train_sentiments)
print(logistic_bow)
logistic_tfidf=lr.fit(tv_train_reviews,train_sentiments)
print(logistic_tfidf)



In [ ]:
logistic_bow_pred = lr.predict(cv_test_reviews)
print(logistic_bow_pred)
logistic_tfidf_pred=lr.predict(tv_test_reviews)
print(logistic_tfidf_pred)


In [ ]:
lr_bow_report=classification_report(test_sentiments,logistic_bow_pred, target_names=['Positive','Negative'])
print(lr_bow_report)
lr_tfidf_report=classification_report(test_sentiments, logistic_tfidf_pred, target_names=['Positive','Negative'])
print(lr_tfidf_report)

In [ ]:
lr_bow_report=classification_report(test_sentiments,logistic_bow_pred, target_names=['Positive','Negative'])
print(lr_bow_report)

lr_tfidf_report=classification_report(test_sentiments, logistic_tfidf_pred, target_names=['Positive', 'Negative'])
print(lr_tfidf_report)


In [ ]:
svm=SGDClassifier(loss='hinge', max_iter=1000, random_state=77)
svm_bow=svm.fit(cv_train_reviews, train_sentiments)
print(svm_bow)
svm_tfidf=svm.fit(tv_train_reviews, train_sentiments)
print(svm_tfidf)





In [ ]:

svm_bow_predict=svm.predict(cv_test_reviews)
print(svm_bow_predict)
#Predicting the model for tfidf features
svm_tfidf_predict=svm.predict(tv_test_reviews)
print(svm_tfidf_predict)

svm_bow_report=classification_report(test_sentiments, svm_bow_predict, target_names=['Positive','Negative'])
print(svm_bow_report)
svm_tfidf_report=classification_report(test_sentiments, svm_tfidf_predict,target_names=['Positive','Negative'])
print(svm_tfidf_report)


In [ ]:
svm_bow_score=accuracy_score(test_sentiments,svm_bow_predict)
print("svm_bow_score :",svm_bow_score)
#Accuracy score for tfidf features
svm_tfidf_score=accuracy_score(test_sentiments,svm_tfidf_predict)
print("svm_tfidf_score :",svm_tfidf_score)

In [ ]:
#Classification report for bag of words 
svm_bow_report=classification_report(test_sentiments,svm_bow_predict,target_names=['Positive','Negative'])
print(svm_bow_report)
#Classification report for tfidf features
svm_tfidf_report=classification_report(test_sentiments,svm_tfidf_predict,target_names=['Positive','Negative'])
print(svm_tfidf_report)

Multinomial Naive Bayes

In [ ]:
#training the model
mnb=MultinomialNB()
#fitting the svm for bag of words
mnb_bow=mnb.fit(cv_train_reviews,train_sentiments)
print(mnb_bow)
#fitting the svm for tfidf features
mnb_tfidf=mnb.fit(tv_train_reviews,train_sentiments)
print(mnb_tfidf)

In [ ]:
#Predicting the model for bag of words
mnb_bow_predict=mnb.predict(cv_test_reviews)
print(mnb_bow_predict)
#Predicting the model for tfidf features
mnb_tfidf_predict=mnb.predict(tv_test_reviews)
print(mnb_tfidf_predict)

In [ ]:
mnb_bow_report=classification_report(test_sentiments,mnb_bow_predict,target_names=['Positive','Negative'])
print(mnb_bow_report)
#Classification report for tfidf features
mnb_tfidf_report=classification_report(test_sentiments,mnb_tfidf_predict,target_names=['Positive','Negative'])
print(mnb_tfidf_report)

Use Word Cloud to show the most popular positive and negative reviews

In [ ]:
plt.figure(figsize=(10,10))
positive_text=train_reviews[1]
WC=WordCloud(width=1000,height=500,max_words=500,min_font_size=5)
positive_words=WC.generate(positive_text)
plt.imshow(positive_words,interpolation='bilinear')
plt.show


In [ ]:
plt.figure(figsize=(10,10))
negative_text=train_reviews[8]
WC=WordCloud(width=1000,height=500,max_words=500,min_font_size=5)
negative_words=WC.generate(negative_text)
plt.imshow(negative_words,interpolation='bilinear')
plt.show